In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Mato Grosso - Desemprego,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH Educacao,Mato Grosso - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Mato Grosso - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - IDH,Mato Grosso - Consumo de Cimento (t)
0,2014,7.495401,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.746518,0.737561,5.676413e+08,2.779340e+09,...,0.782759,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375,0.755153,1411.234429
1,2015,7.838256,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.758124,0.732393,5.179392e+08,2.561731e+09,...,0.791129,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487,0.759897,1241.100343
2,2016,7.730867,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.757317,0.732701,4.524144e+08,2.516859e+09,...,0.788868,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712,0.758979,1083.128229
3,2017,7.251143,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.756528,0.733401,4.213689e+08,2.497710e+09,...,0.786038,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939,0.758068,1220.058000
4,2018,7.109465,7.676559e+07,2.824325e+06,17.858303,5.618080e+07,0.756485,0.733438,4.166580e+08,2.797145e+09,...,0.782587,26.864064,986.130627,790.527175,27.131978,18.283627,63.192140,1.527134,0.756983,1316.386000
5,2019,7.037991,7.645988e+07,2.828969e+06,17.844789,5.610350e+07,0.755802,0.732478,4.462625e+08,3.484252e+09,...,0.778458,27.060953,988.002188,1086.573235,27.719471,17.709442,59.441754,1.529612,0.755072,1446.064000
6,2020,6.871142,7.584496e+07,2.832984e+06,17.835880,5.599389e+07,0.754420,0.731976,5.489412e+08,4.114911e+09,...,0.773591,26.703448,989.342831,1296.495701,27.842739,17.441574,57.842076,1.547261,0.752897,1555.266000
7,2021,6.623657,7.372752e+07,2.820526e+06,17.690471,5.530452e+07,0.753458,0.731701,6.638811e+08,4.421848e+09,...,0.774008,25.957132,989.869529,1388.381414,27.804463,17.298090,57.234541,1.600142,0.752615,1788.146000
8,2022,6.630925,7.119278e+07,2.804678e+06,17.513528,5.446856e+07,0.753045,0.732764,7.010332e+08,4.533216e+09,...,0.778009,25.101402,989.986809,1412.105050,27.774281,17.132150,56.722192,1.639477,0.754125,1947.017000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Mato Grosso - Desemprego,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH Educacao,Mato Grosso - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - IDH
0,7.495401,7.450695e+07,2.898181e+06,17.765377,5.559725e+07,0.746518,0.737561,5.676413e+08,2.779340e+09,1.954361e+10,...,1268.814714,0.782759,33.403862,985.582958,728.555092,26.792583,18.998037,66.249639,1.689375,0.755153
1,7.838256,7.552129e+07,2.867943e+06,17.804826,5.583983e+07,0.758124,0.732393,5.179392e+08,2.561731e+09,1.327596e+10,...,1123.345627,0.791129,31.029765,985.297803,631.112819,26.741398,18.889658,66.109773,1.670487,0.759897
2,7.730867,7.631332e+07,2.837856e+06,17.844276,5.605555e+07,0.757317,0.732701,4.524144e+08,2.516859e+09,1.011370e+10,...,1075.219903,0.788868,27.875013,985.139748,546.211949,26.647262,18.703122,65.821838,1.617712,0.758979
3,7.251143,7.659073e+07,2.826355e+06,17.841399,5.610212e+07,0.756528,0.733401,4.213689e+08,2.497710e+09,8.456309e+09,...,1111.839984,0.786038,26.025417,985.051490,572.018116,26.728560,18.583504,65.279143,1.559939,0.758068
4,7.109465,7.676559e+07,2.824325e+06,17.858303,5.618080e+07,0.756485,0.733438,4.166580e+08,2.797145e+09,7.937760e+09,...,1139.872508,0.782587,26.864064,986.130627,790.527175,27.131978,18.283627,63.192140,1.527134,0.756983
5,7.037991,7.645988e+07,2.828969e+06,17.844789,5.610350e+07,0.755802,0.732478,4.462625e+08,3.484252e+09,1.091282e+10,...,1211.033241,0.778458,27.060953,988.002188,1086.573235,27.719471,17.709442,59.441754,1.529612,0.755072
6,6.871142,7.584496e+07,2.832984e+06,17.835880,5.599389e+07,0.754420,0.731976,5.489412e+08,4.114911e+09,1.625333e+10,...,1314.665925,0.773591,26.703448,989.342831,1296.495701,27.842739,17.441574,57.842076,1.547261,0.752897


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    1241.100343
1    1083.128229
2    1220.058000
3    1316.386000
4    1446.064000
5    1555.266000
6    1788.146000
7    1947.017000
8            NaN
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso - Desemprego,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH Educacao,Mato Grosso - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - IDH
0,0.280964,-1.935048,1.878517,-1.918446,-1.903705,-2.197184,2.176396,1.779974,0.019134,2.003355,...,1.743739,-0.523797,1.786018,-0.279274,0.014938,-0.449727,1.090481,0.785054,1.395692,-1.214014
1,1.415860,-0.643175,0.762755,-0.680159,-0.696662,0.764226,-0.708428,0.870373,-0.621387,0.401153,...,-0.485383,1.456092,0.882702,-0.559180,-0.519818,-0.587053,0.839098,0.727281,1.103869,1.397241
2,1.060387,0.365558,-0.347421,0.558122,0.376690,0.558345,-0.536271,-0.328802,-0.753466,-0.407217,...,-1.222847,0.921288,-0.317644,-0.714326,-0.985747,-0.839610,0.406429,0.608347,0.288448,0.891718
3,-0.527560,0.718871,-0.771799,0.467824,0.608437,0.356943,-0.145496,-0.896969,-0.809829,-0.830898,...,-0.661692,0.251852,-1.021394,-0.800958,-0.844125,-0.621494,0.128977,0.384183,-0.604188,0.390276
4,-0.996532,0.941574,-0.846713,0.998414,0.999943,0.345993,-0.125130,-0.983185,0.071542,-0.963455,...,-0.232131,-0.564454,-0.702298,0.258314,0.355036,0.460843,-0.566585,-0.477869,-1.111054,-0.206789
5,-1.233119,0.552220,-0.675338,0.574244,0.615298,0.171678,-0.661072,-0.441390,2.094006,-0.202939,...,0.858314,-1.540982,-0.627384,2.095424,1.979715,2.037041,-1.898401,-2.026996,-1.072767,-1.258432


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    1241.100343
1    1083.128229
2    1220.058000
3    1316.386000
4    1446.064000
5    1555.266000
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso - Desemprego,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH Educacao,Mato Grosso - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - IDH
6,-1.370174,-0.213009,-0.478695,0.271047,0.064717,-0.167048,-0.821048,1.16956,2.035192,0.985175,...,1.66295,-1.747087,-0.679077,1.935585,1.871142,1.633502,-1.689288,-1.745637,-0.709021,-1.666374


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    1788.146
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [33]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(148, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(148, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [34]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [35]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3902339896, 3931747110, 827925262, 542531193, 1912873603, 2337924107, 4081917260, 3888145535, 2975601758, 2548464241]


Step: 0 ___________________________________________
Epoch 1198: early stopping
val_loss: 0.09598499536514282
winner_seed: 3902339896


Step: 1 ___________________________________________
Epoch 1642: early stopping
val_loss: 0.1458919197320938


Step: 2 ___________________________________________
Epoch 1971: early stopping
val_loss: 508.3120422363281


Step: 3 ___________________________________________
Epoch 1291: early stopping
val_loss: 0.0366830974817276
winner_seed: 542531193


Step: 4 ___________________________________________
Epoch 1585: early stopping
val_loss: 340.7972106933594


Step: 5 ___________________________________________
Epoch 1313: early stopping
val_loss: 58290.35546875


Step: 6 ___________________________________________
Epoch 2083: early stopping
val_loss: 0.0006140619516372681
winner_seed: 4081917260


Step: 7 ________________________________

In [36]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 301ms/step - loss: 1740745.6250 - val_loss: 2396003.5000
Epoch 2/5000
1/1 [==============================] - 0s 12ms/step - loss: 1724060.6250 - val_loss: 2491344.2500
Epoch 3/5000
1/1 [==============================] - 0s 11ms/step - loss: 1727006.6250 - val_loss: 2349475.2500
Epoch 4/5000
1/1 [==============================] - 0s 11ms/step - loss: 1679854.0000 - val_loss: 2227873.0000
Epoch 5/5000
1/1 [==============================] - 0s 11ms/step - loss: 1672316.6250 - val_loss: 2393423.2500
Epoch 6/5000
1/1 [==============================] - 0s 12ms/step - loss: 1671711.0000 - val_loss: 2364543.2500
Epoch 7/5000
1/1 [==============================] - 0s 12ms/step - loss: 1655247.5000 - val_loss: 2492931.7500
Epoch 8/5000
1/1 [==============================] - 0s 11ms/step - loss: 1675555.8750 - val_loss: 2456664.7500
Epoch 9/5000
1/1 [==============================] - 0s 11ms/step - loss: 1649176.0000 - val_loss: 2384739.2500


1/1 [==============================] - 0s 12ms/step - loss: 1187366.8750 - val_loss: 1545749.5000
Epoch 75/5000
1/1 [==============================] - 0s 12ms/step - loss: 1231713.1250 - val_loss: 1535139.5000
Epoch 76/5000
1/1 [==============================] - 0s 12ms/step - loss: 1156407.1250 - val_loss: 2098750.7500
Epoch 77/5000
1/1 [==============================] - 0s 11ms/step - loss: 1316884.6250 - val_loss: 2125202.2500
Epoch 78/5000
1/1 [==============================] - 0s 11ms/step - loss: 1270394.8750 - val_loss: 1532276.5000
Epoch 79/5000
1/1 [==============================] - 0s 11ms/step - loss: 1065319.8750 - val_loss: 1521358.3750
Epoch 80/5000
1/1 [==============================] - 0s 11ms/step - loss: 1111335.6250 - val_loss: 1620474.0000
Epoch 81/5000
1/1 [==============================] - 0s 11ms/step - loss: 1172968.0000 - val_loss: 1611228.2500
Epoch 82/5000
1/1 [==============================] - 0s 12ms/step - loss: 1101512.1250 - val_loss: 1613241.1250
Epoch 

1/1 [==============================] - 0s 11ms/step - loss: 675964.6250 - val_loss: 1163864.1250
Epoch 148/5000
1/1 [==============================] - 0s 11ms/step - loss: 778571.5000 - val_loss: 652507.3750
Epoch 149/5000
1/1 [==============================] - 0s 11ms/step - loss: 630389.5625 - val_loss: 642947.7500
Epoch 150/5000
1/1 [==============================] - 0s 11ms/step - loss: 762852.5625 - val_loss: 355734.9375
Epoch 151/5000
1/1 [==============================] - 0s 11ms/step - loss: 714162.1875 - val_loss: 350551.3750
Epoch 152/5000
1/1 [==============================] - 0s 11ms/step - loss: 749060.1875 - val_loss: 854151.4375
Epoch 153/5000
1/1 [==============================] - 0s 11ms/step - loss: 740739.4375 - val_loss: 1043514.3750
Epoch 154/5000
1/1 [==============================] - 0s 11ms/step - loss: 670364.0000 - val_loss: 1033421.6250
Epoch 155/5000
1/1 [==============================] - 0s 11ms/step - loss: 607430.1875 - val_loss: 1024611.2500
Epoch 156/50

Epoch 221/5000
1/1 [==============================] - 0s 11ms/step - loss: 582135.1250 - val_loss: 561581.5625
Epoch 222/5000
1/1 [==============================] - 0s 11ms/step - loss: 594227.6875 - val_loss: 556214.6250
Epoch 223/5000
1/1 [==============================] - 0s 11ms/step - loss: 627108.6875 - val_loss: 559509.3750
Epoch 224/5000
1/1 [==============================] - 0s 11ms/step - loss: 589843.1250 - val_loss: 557391.1250
Epoch 225/5000
1/1 [==============================] - 0s 11ms/step - loss: 618546.6250 - val_loss: 561807.3125
Epoch 226/5000
1/1 [==============================] - 0s 11ms/step - loss: 600713.1875 - val_loss: 566009.9375
Epoch 227/5000
1/1 [==============================] - 0s 11ms/step - loss: 461203.6562 - val_loss: 566126.9375
Epoch 228/5000
1/1 [==============================] - 0s 11ms/step - loss: 606146.5625 - val_loss: 711020.1250
Epoch 229/5000
1/1 [==============================] - 0s 11ms/step - loss: 651490.0625 - val_loss: 712194.4375
E

Epoch 295/5000
1/1 [==============================] - 0s 12ms/step - loss: 504180.5938 - val_loss: 907974.3750
Epoch 296/5000
1/1 [==============================] - 0s 11ms/step - loss: 431932.8438 - val_loss: 909877.3125
Epoch 297/5000
1/1 [==============================] - 0s 12ms/step - loss: 504884.0938 - val_loss: 909554.5625
Epoch 298/5000
1/1 [==============================] - 0s 11ms/step - loss: 469680.1562 - val_loss: 909488.1875
Epoch 299/5000
1/1 [==============================] - 0s 11ms/step - loss: 521324.9688 - val_loss: 912273.8125
Epoch 300/5000
1/1 [==============================] - 0s 11ms/step - loss: 468352.3438 - val_loss: 915863.9375
Epoch 301/5000
1/1 [==============================] - 0s 12ms/step - loss: 506456.0938 - val_loss: 909864.9375
Epoch 302/5000
1/1 [==============================] - 0s 12ms/step - loss: 416909.4688 - val_loss: 913412.3125
Epoch 303/5000
1/1 [==============================] - 0s 11ms/step - loss: 492222.0000 - val_loss: 910401.5625
E

...

1/1 [==============================] - 0s 12ms/step - loss: 19851.0098 - val_loss: 233.0281
Epoch 1711/5000
1/1 [==============================] - 0s 12ms/step - loss: 17905.7480 - val_loss: 277.6056
Epoch 1712/5000
1/1 [==============================] - 0s 11ms/step - loss: 960.1682 - val_loss: 278.2486
Epoch 1713/5000
1/1 [==============================] - 0s 11ms/step - loss: 2199.8806 - val_loss: 285.3097
Epoch 1714/5000
1/1 [==============================] - 0s 11ms/step - loss: 6665.1323 - val_loss: 312.0881
Epoch 1715/5000
1/1 [==============================] - 0s 11ms/step - loss: 11481.2969 - val_loss: 357.9773
Epoch 1716/5000
1/1 [==============================] - 0s 11ms/step - loss: 24079.0137 - val_loss: 432.1141
Epoch 1717/5000
1/1 [==============================] - 0s 11ms/step - loss: 1088.4573 - val_loss: 429.5246
Epoch 1718/5000
1/1 [==============================] - 0s 11ms/step - loss: 1825.7061 - val_loss: 436.1581
Epoch 1719/5000
1/1 [=============================

1/1 [==============================] - 0s 11ms/step - loss: 7352.0688 - val_loss: 25101.9766
Epoch 1786/5000
1/1 [==============================] - 0s 11ms/step - loss: 27153.4980 - val_loss: 25302.2383
Epoch 1787/5000
1/1 [==============================] - 0s 11ms/step - loss: 43619.5273 - val_loss: 25333.4316
Epoch 1788/5000
1/1 [==============================] - 0s 11ms/step - loss: 16709.3027 - val_loss: 25626.1777
Epoch 1789/5000
1/1 [==============================] - 0s 11ms/step - loss: 24036.5527 - val_loss: 26305.1406
Epoch 1790/5000
1/1 [==============================] - 0s 11ms/step - loss: 14971.8828 - val_loss: 26022.7461
Epoch 1791/5000
1/1 [==============================] - 0s 11ms/step - loss: 6374.0171 - val_loss: 25727.1504
Epoch 1792/5000
1/1 [==============================] - 0s 11ms/step - loss: 11987.4229 - val_loss: 25399.2617
Epoch 1793/5000
1/1 [==============================] - 0s 11ms/step - loss: 24670.5938 - val_loss: 25578.8711
Epoch 1794/5000
1/1 [=======

Epoch 1860/5000
1/1 [==============================] - 0s 11ms/step - loss: 17549.8789 - val_loss: 25017.2988
Epoch 1861/5000
1/1 [==============================] - 0s 12ms/step - loss: 6838.7559 - val_loss: 24858.7266
Epoch 1862/5000
1/1 [==============================] - 0s 11ms/step - loss: 7647.4297 - val_loss: 24705.6074
Epoch 1863/5000
1/1 [==============================] - 0s 11ms/step - loss: 9017.9238 - val_loss: 24959.6035
Epoch 1864/5000
1/1 [==============================] - 0s 11ms/step - loss: 6373.0815 - val_loss: 24876.8594
Epoch 1865/5000
1/1 [==============================] - 0s 11ms/step - loss: 5393.0073 - val_loss: 24584.8398
Epoch 1866/5000
1/1 [==============================] - 0s 11ms/step - loss: 22243.3203 - val_loss: 24982.2480
Epoch 1867/5000
1/1 [==============================] - 0s 11ms/step - loss: 9257.9209 - val_loss: 25085.6953
Epoch 1868/5000
1/1 [==============================] - 0s 12ms/step - loss: 31466.2598 - val_loss: 25444.8848
Epoch 1869/5000


Epoch 1935/5000
1/1 [==============================] - 0s 11ms/step - loss: 5976.6987 - val_loss: 22052.1777
Epoch 1936/5000
1/1 [==============================] - 0s 11ms/step - loss: 16959.2676 - val_loss: 22422.3945
Epoch 1937/5000
1/1 [==============================] - 0s 12ms/step - loss: 9133.4766 - val_loss: 22285.0371
Epoch 1938/5000
1/1 [==============================] - 0s 11ms/step - loss: 20327.4629 - val_loss: 22537.8457
Epoch 1939/5000
1/1 [==============================] - 0s 11ms/step - loss: 24918.4512 - val_loss: 22775.7910
Epoch 1940/5000
1/1 [==============================] - 0s 11ms/step - loss: 15214.6592 - val_loss: 23098.2852
Epoch 1941/5000
1/1 [==============================] - 0s 12ms/step - loss: 14471.7109 - val_loss: 23519.8086
Epoch 1942/5000
1/1 [==============================] - 0s 12ms/step - loss: 7824.5308 - val_loss: 23379.1270
Epoch 1943/5000
1/1 [==============================] - 0s 11ms/step - loss: 13830.8838 - val_loss: 23615.8340
Epoch 1944/50

Epoch 2010/5000
1/1 [==============================] - 0s 11ms/step - loss: 11970.0469 - val_loss: 6136.3853
Epoch 2011/5000
1/1 [==============================] - 0s 12ms/step - loss: 17971.7969 - val_loss: 6344.8457
Epoch 2012/5000
1/1 [==============================] - 0s 11ms/step - loss: 6879.3335 - val_loss: 6208.9858
Epoch 2013/5000
1/1 [==============================] - 0s 11ms/step - loss: 13725.8213 - val_loss: 6345.6821
Epoch 2014/5000
1/1 [==============================] - 0s 11ms/step - loss: 11422.8945 - val_loss: 6387.1157
Epoch 2015/5000
1/1 [==============================] - 0s 11ms/step - loss: 9905.7383 - val_loss: 6486.4619
Epoch 2016/5000
1/1 [==============================] - 0s 11ms/step - loss: 25832.7480 - val_loss: 6681.2466
Epoch 2017/5000
1/1 [==============================] - 0s 11ms/step - loss: 7698.5483 - val_loss: 6663.8564
Epoch 2018/5000
1/1 [==============================] - 0s 11ms/step - loss: 6714.7886 - val_loss: 6518.5713
Epoch 2019/5000
1/1 [==

In [37]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 33ms/step
[1788.146](test_target) - [[1623.4236]](prediction) = 164.72241601562496


In [38]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [39]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso - Desemprego,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH Educacao,Mato Grosso - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - IDH
0,-1.346294,-1.271782,1.225768,-1.224746,-1.247964,-1.411470,1.412211,1.165029,1.396038,1.335266,...,1.373322,-1.365166,1.163243,1.323797,1.251837,1.088353,1.103707,1.062439,0.995264,-1.390461
1,1.048140,0.100234,-0.002049,0.000002,0.047841,0.781981,-0.771257,0.111769,-0.502296,-0.264151,...,-0.394273,1.002336,0.114905,-0.231004,-0.056114,0.237882,0.213896,0.277123,0.372476,0.918756
2,0.298154,1.171548,-1.223719,1.224744,1.200124,0.629490,-0.640954,-1.276798,-0.893743,-1.071116,...,-0.979049,0.362830,-1.278148,-1.092794,-1.195724,-1.326235,-1.317603,-1.339562,-1.367740,0.471706


train_target:


,Mato Grosso - Consumo de Cimento (t)
0,1241.100343
1,1083.128229
2,1220.058000


test_input:


...

,Mato Grosso - Desemprego,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - IDH Educacao,Mato Grosso - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - IDH
6,-1.370174,-0.213009,-0.478695,0.271047,0.064717,-0.167048,-0.821048,1.16956,2.035192,0.985175,...,1.66295,-1.747087,-0.679077,1.935585,1.871142,1.633502,-1.689288,-1.745637,-0.709021,-1.666374


test_target:


,Mato Grosso - Consumo de Cimento (t)
6,1788.146


Epoch 2083: early stopping
1/1 [==============================] - 0s 33ms/step
Error: 164.72241601562496




[1209.72216796875, 1268.1629638671875, 1274.5045166015625, 1623.423583984375]

In [40]:
display(targets)
display(predictions)

[1316.386, 1446.064, 1555.266, 1788.146]

[1209.72216796875, 1268.1629638671875, 1274.5045166015625, 1623.423583984375]

In [41]:
mae = mean_absolute_error(predictions, targets)
mae

182.51219189453127

In [42]:
porcentage = mae/np.mean(targets)
porcentage

0.11956522561075325